Import

In [1]:
import pandas as pd

# Load the CSV file
input_file = 's_aes_Quadruplets.csv'  # Replace with your input file name
df = pd.read_csv(input_file)

# Select the first 1000 records
df_first_10000 = df.head(10000)

# Save the selected records to another CSV file
output_file = 'output_first_10000.csv'  # Replace with your desired output file name
df_first_10000.to_csv(output_file, index=False)

print(f"First 1000 records saved to {output_file}")


First 1000 records saved to output_first_10000.csv


In [117]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, SimpleRNN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

#from keras.layers import SimpleRNN, Dense

Upload the data

In [3]:
data = pd.read_csv('output_first_10000.csv')

Data prepration

In [118]:

# Convert all hexadecimal entries to binary representation in the entire dataframe

# y = data[['key']]
# y2= data[['iv']]
# print(y2)
y=data.drop(['plaintext','ciphertext','before_xor'], axis=1)
x = data.drop(['key','before_xor','iv'], axis=1)
def convert_to_binary(hex_number):
    return bin(int(hex_number, 16))[2:].zfill(16)

x = x.applymap(convert_to_binary)
y = y.applymap(convert_to_binary)




X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

plaintext_train = np.array([list(map(int, list(row))) for row in X_train['plaintext'].values])
ciphertext_train = np.array([list(map(int, list(row))) for row in X_train['ciphertext'].values])

plaintext_test = np.array([list(map(int, list(row))) for row in X_test['plaintext'].values])
ciphertext_test = np.array([list(map(int, list(row))) for row in X_test['ciphertext'].values])

# Stack the arrays to create the transformed datasets
X_train_transformed = np.stack([plaintext_train, ciphertext_train], axis=-1)
X_test_transformed = np.stack([plaintext_test, ciphertext_test], axis=-1)

# For the y_train and y_test
key_train = np.array([list(map(int, list(row))) for row in y_train['key'].values])
key_test = np.array([list(map(int, list(row))) for row in y_test['key'].values])
iv_train = np.array([list(map(int, list(row))) for row in y_train['iv'].values])
iv_test = np.array([list(map(int, list(row))) for row in y_test['iv'].values])

y_train_transformed = np.stack([key_train, iv_train], axis=-1)
y_test_transformed = np.stack([key_test, iv_test], axis=-1)
# arr = X_train.apply(lambda row: [[int(row['plaintext'][i]), int(row['ciphertext'][i]), int(row['iv'][i])] for i in range(16)], axis=1).to_numpy()

# X_train_transformed = np.zeros((X_train.shape[0], 16, 3), dtype=int)
# for i in range(X_train.shape[0]):
#     for j in range(16):
#         X_train_transformed[i, j, :] = arr[i][j]

# arr = X_test.apply(lambda row: [[int(row['plaintext'][i]), int(row['ciphertext'][i]), int(row['iv'][i])] for i in range(16)], axis=1).to_numpy()

# X_test_transformed = np.zeros((X_test.shape[0], 16, 3), dtype=int)
# for i in range(X_test.shape[0]):
#     for j in range(16):
#         X_test_transformed[i, j, :] = arr[i][j]

# arr = y_train.apply(lambda row: [[int(row['key'][i])] for i in range(16)], axis=1).to_numpy()

# y_train_transformed = np.zeros((y_train.shape[0], 16,1 ), dtype=int)
# for i in range(y_train.shape[0]):
#     for j in range(16):
#         y_train_transformed[i, j, :] = arr[i][j]



# y_test_transformed = np.zeros((y_test.shape[0], 16,1 ), dtype=int)
# for i in range(y_test.shape[0]):
#     for j in range(16):
#         y_test_transformed[i, j, :] = arr[i][j]







C:\Users\omara\AppData\Local\Temp\ipykernel_21000\1478871397.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  x = x.applymap(convert_to_binary)
C:\Users\omara\AppData\Local\Temp\ipykernel_21000\1478871397.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  y = y.applymap(convert_to_binary)


In [5]:
y_test.shape
print(y_test_transformed[0])

[[1 0]
 [1 0]
 [1 1]
 [0 0]
 [1 1]
 [0 0]
 [1 0]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [1 0]
 [0 0]
 [0 0]
 [1 0]
 [0 1]]


Build the model

In [6]:
# Define the model
input_layer = layers.Input(shape=(16, 2))

# First Dense layer
dense1 = layers.Dense(256, activation='relu')(input_layer)

# Bidirectional RNN layer
bidirectional_rnn = layers.Bidirectional(layers.SimpleRNN(200))(dense1)

# Second Dense layer
dense2 = layers.Dense(256, activation='relu')(bidirectional_rnn)
dense3 = layers.Dense(256, activation='relu')(dense2)

# Final Dense layer with Sigmoid activation for IVs
ivs_out = layers.Dense(16, activation='sigmoid',name='ivs_out')(dense3)

ivs_out_reshaped = layers.Reshape((16, 1))(ivs_out)

concatenated = layers.Concatenate(axis=-1)([input_layer, ivs_out_reshaped])  # Concatenate along the last axis

dense4 = layers.Dense(256, activation='relu')(concatenated)

# # Bidirectional RNN on reshaped input
bidirectional_rnn2 = layers.Bidirectional(layers.SimpleRNN(200))(dense4)

# # Second Dense layer for key_out
dense5 = layers.Dense(256, activation='relu')(bidirectional_rnn2)
dense6 = layers.Dense(256, activation='relu')(dense5)

# # Final Dense layer with Sigmoid activation for Key
key_out = layers.Dense(16, activation='sigmoid',name='key_out')(dense6)

# Define the model
model = Model(inputs=input_layer, outputs=[ivs_out,key_out])

optimizer = Adam(learning_rate=0.00001)

# Compile the model
reduce_lr = ReduceLROnPlateau(
    monitor='loss',  # Metric to monitor
    factor=0.5,          # Factor by which to reduce the learning rate
    patience=5,          # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-9          # Lower bound on the learning rate
)
model.compile(optimizer=optimizer,
              loss={'ivs_out': 'binary_crossentropy', 'key_out': 'binary_crossentropy'},
              metrics={'ivs_out': ['accuracy'], 'key_out': ['accuracy']})

In [7]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 16, 2)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 16, 256)   │        768 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 400)       │    182,800 │ dense[0][0]       │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │    102,656 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │     65,792 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ivs_out (Dense)     │ (None, 16)        │      4,112 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 16, 1)     │          0 │ ivs_out[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 16, 3)     │          0 │ input_layer[0][0… │
│ (Concatenate)       │                   │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 16, 256)   │      1,024 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 400)       │    182,800 │ dense_3[0][0]     │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256)       │    102,656 │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 256)       │     65,792 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ key_out (Dense)     │ (None, 16)        │      4,112 │ dense_5[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 712,512 (2.72 MB)

 Trainable params: 712,512 (2.72 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
y_key,y_ivs = y_train_transformed[:, :,0], y_train_transformed[:, :,1]
print(y_ivs.shape)
print(y_key.shape)
print(y_train_transformed.shape)

(9000, 16)
(9000, 16)
(9000, 16, 2)


In [9]:
print(X_train_transformed.shape)
print(type(y_ivs))
print(type(y_key))
print(X_train_transformed.shape)


(9000, 16, 2)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(9000, 16, 2)


In [11]:
model.fit(X_train_transformed, [y_ivs, y_key], epochs=2000, batch_size=128, callbacks=[reduce_lr], verbose=1)


Epoch 1/2000
71/71 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - ivs_out_accuracy: 0.1534 - ivs_out_loss: 0.0425 - key_out_accuracy: 0.1513 - key_out_loss: 0.0078 - loss: 0.0503 - learning_rate: 1.0000e-05
Epoch 2/2000
71/71 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - ivs_out_accuracy: 0.1444 - ivs_out_loss: 0.0425 - key_out_accuracy: 0.1550 - key_out_loss: 0.0078 - loss: 0.0503 - learning_rate: 1.0000e-05
Epoch 3/2000
71/71 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - ivs_out_accuracy: 0.1495 - ivs_out_loss: 0.0427 - key_out_accuracy: 0.1571 - key_out_loss: 0.0078 - loss: 0.0505 - learning_rate: 1.0000e-05
Epoch 4/2000
71/71 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - ivs_out_accuracy: 0.1500 - ivs_out_loss: 0.0423 - key_out_accuracy: 0.1529 - key_out_loss: 0.0078 - loss: 0.0501 - learning_rate: 1.0000e-05
Epoch 5/2000
71/71 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - ivs_out_accuracy: 0.1506 - ivs_out_loss: 0.0420 - key_out_accuracy: 0.1555 - key_out_loss: 0.0077 - loss: 0.0497 - learning_rate: 1.0000e-05
Epoch 6/2000
71/71 ━

Evaluate the model (training accuracy)

In [119]:
#model = tf.keras.models.load_model('models/test_checkpoint_rnn.h5')


In [120]:

# Evaluate the model
y_keyt,y_ivst = y_test_transformed[:, :,0], y_test_transformed[:, :,1]

metrics = model.evaluate(X_test_transformed, [y_ivst,y_keyt], verbose=0)
print(f"Model Evaluation Accuracy: {metrics}")

# Make predictions
y_pred = (np.array(model.predict(X_train_transformed)) > 0.5).astype("int32")

# # Calculate accuracy of predictions
# y_test_transformed=y_test_transformed.reshape(y_test_transformed.shape[0],16)
# accuracy = accuracy_score(np.ravel(y_test_transformed), np.ravel(y_pred))
# print(f"Accuracy Score: {accuracy}")
# # accuracy = accuracy_score(y_test_transformed, y_pred)
# # print(f"Accuracy Score: {accuracy}")



Model Evaluation Accuracy: [9.278233528137207, 3.647246837615967, 5.5983405113220215, 0.07100000232458115, 0.08500000089406967]
282/282 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


In [121]:
y_pred.shape
y_pred_swapped = y_pred.copy() 
y_pred_swapped[0], y_pred_swapped[1] = y_pred[1], y_pred[0]


In [123]:
correct_count_ivs = 0
correct_count_key = 0
correct_count = 0

# Iterate over each array
for i in range(y_train_transformed.shape[0]):
    if np.array_equal(y_train_transformed[i,:,0], y_pred_swapped[0,i,:]):
        correct_count_ivs += 1
    if np.array_equal(y_train_transformed[i,:,1], y_pred_swapped[1,i,:]):
        correct_count_key += 1
    if np.array_equal(y_train_transformed[i,:,0], y_pred_swapped[0,i,:]) and np.array_equal(y_train_transformed[i,:,1], y_pred_swapped[1,i,:]):
        correct_count += 1
   
       

# Calculate accuracy
total_arrays = y_train_transformed.shape[0]
accuracy = correct_count / total_arrays

print(f"Number of correct IVs predictions: {correct_count_ivs}")
print(f"Number of correct Key predictions: {correct_count_key}")

print(f"Number of correct predictions: {correct_count}")
print(f"Accuracy: {accuracy * 100:.2f}%")


Number of correct IVs predictions: 9000
Number of correct Key predictions: 9000
Number of correct predictions: 9000
Accuracy: 100.00%


(validation_accuracy)

In [124]:

# Evaluate the model

# Make predictions
y_pred2 = (np.array(model.predict(X_test_transformed)) > 0.5).astype("int32")



32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [125]:
y_pred2.shape
y_pred_swapped2 = y_pred.copy() 
y_pred_swapped2[0], y_pred_swapped[1] = y_pred[1], y_pred[0]


In [126]:
correct_count_ivs = 0
correct_count_key = 0
correct_count = 0

# Iterate over each array
for i in range(y_test_transformed.shape[0]):
    if np.array_equal(y_test_transformed[i,:,0], y_pred_swapped2[0,i,:]):
        correct_count_ivs +=1
    if np.array_equal(y_test_transformed[i,:,1], y_pred_swapped2[1,i,:]):
        correct_count_key += 1
    if np.array_equal(y_test_transformed[i,:,0], y_pred_swapped2[0,i,:]) and np.array_equal(y_test_transformed[i,:,1], y_pred_swapped2[1,i,:]):
        correct_count += 1
   
       

# Calculate accuracy
total_arrays = y_train_transformed.shape[0]
accuracy = correct_count / total_arrays

print(f"Number of correct IVs predictions: {correct_count_ivs}")
print(f"Number of correct Key predictions: {correct_count_key}")

print(f"Number of correct predictions: {correct_count}")
print(f"Accuracy: {accuracy * 100:.2f}%")

Number of correct IVs predictions: 2
Number of correct Key predictions: 1
Number of correct predictions: 0
Accuracy: 0.00%
